In [1]:
import pandas as pd
import os
import glob
from tqdm import tqdm
import math
from rake_nltk import Rake
from transformers import pipeline
import warnings

import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import time

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.cluster import KMeans

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

/home/krish/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/krish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/krish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/krish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
df = pd.read_csv("reviews/e2w.csv", names=["reviews", "attributes", "ratings"])

In [14]:
df.head()

,reviews,attributes,ratings
0,Wanna buy this bike so i gave basic informatio...,"['Everything', 'Never owned', '']",1
1,This is simply amazing and exceptionally well ...,"['Everything', '> 1 yr', '< 5000 kms', '']",5
2,"Battery life is good, I like this bike very mu...","['Everything', '< 3 months', '< 5000 kms', '']",5
3,I seen this bike on road. Very nice bike and g...,"['Everything', 'Never owned', '']",5
4,I have a rv400 I have the bike for almost more...,"['Daily Commute', '> 1 yr', '5000-10000 kms', '']",1


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviews     800 non-null    object
 1   attributes  800 non-null    object
 2   ratings     800 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 18.9+ KB


In [16]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviews     799 non-null    object
 1   attributes  799 non-null    object
 2   ratings     799 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 18.9+ KB


In [18]:
df["usage"] = np.zeros(len(df))
df["duration"] = np.zeros(len(df))
df["km_driven"] = np.zeros(len(df))

In [19]:
for i in range(len(df)):
    df.loc[i, "usage"] = df["attributes"][i].replace("'", '').replace("[", '').replace("]", '').split(',')[0]
    df.loc[i, "duration"] = df["attributes"][i].replace("'", '').replace("[", '').replace("]", '').split(',')[1]    
    df.loc[i, "km_driven"] = df["attributes"][i].replace("'", '').replace("[", '').replace("]", '').split(',')[2]    
df.drop(["attributes"], axis=1, inplace=True)

In [20]:
df.head(10)

,reviews,ratings,usage,duration,km_driven
0,Wanna buy this bike so i gave basic informatio...,1,Everything,Never owned,
1,This is simply amazing and exceptionally well ...,5,Everything,> 1 yr,< 5000 kms
2,"Battery life is good, I like this bike very mu...",5,Everything,< 3 months,< 5000 kms
3,I seen this bike on road. Very nice bike and g...,5,Everything,Never owned,
4,I have a rv400 I have the bike for almost more...,1,Daily Commute,> 1 yr,5000-10000 kms
5,Buying experience of Revolt 400 bike. It is us...,3,Leisure Rides,> 1 yr,> 15000 kms
6,"We'll, this bike will meet the interests of ma...",5,Leisure Rides,Never owned,
7,"Buying experience is awesome, riding performan...",5,Daily Commute,6 months-1 yr,< 5000 kms
8,Value for money. Really a thumbs up for the bi...,5,Everything,Never owned,
9,"Please give protection to the battery, because...",4,Daily Commute,Never owned,


In [21]:
for col in df.columns[1:].values:
    print(col,'\n\n' ,df[col].value_counts(), '\n\n\n')

ratings 

 5    321
1    227
4    144
3     64
2     43
Name: ratings, dtype: int64 



usage 

 Daily Commute         566
Everything            154
Occasional Commute     52
Leisure Rides          19
Tours                   8
Name: usage, dtype: int64 



duration 

  < 3 months            209
 > 1 yr                192
 Never owned           172
 6 months-1 yr         126
 3-6 months             99
 Occasional Commute      1
Name: duration, dtype: int64 



km_driven 

  < 5000 kms         430
                    173
 5000-10000 kms     129
 > 15000 kms         36
 10000-15000 kms     30
 Never owned          1
Name: km_driven, dtype: int64 





In [22]:
from sklearn.preprocessing import OrdinalEncoder

In [23]:
ordinal = OrdinalEncoder()

In [24]:
columns = df.columns[2:].values
df[columns] = ordinal.fit_transform(df[columns])

In [25]:
df

,reviews,ratings,usage,duration,km_driven
0,Wanna buy this bike so i gave basic informatio...,1,1.0,4.0,0.0
1,This is simply amazing and exceptionally well ...,5,1.0,3.0,3.0
2,"Battery life is good, I like this bike very mu...",5,1.0,2.0,3.0
3,I seen this bike on road. Very nice bike and g...,5,1.0,4.0,0.0
4,I have a rv400 I have the bike for almost more...,1,0.0,3.0,2.0
...,...,...,...,...,...
794,I brought this about 2 year back. usage are le...,1,0.0,3.0,2.0
795,Purchased on april 2019. After 3 month lock go...,3,0.0,1.0,3.0
796,Got it delivered to me from pune...Doesn't fee...,5,0.0,3.0,2.0
797,"Bought 2W in 2019 for 55,000-60,000/-. Have cl...",4,0.0,3.0,2.0


In [26]:
start = [0, 0, 0, 0, 0]

In [28]:
start[0]+=1

In [29]:
start

[1, 0, 0, 0, 0]

In [30]:
end =[]
start = []

In [36]:
my_dict = {"one":[] , "two": []}

In [38]:
my_dict["one"].append(1)

In [39]:
my_dict

{'one': [1], 'two': []}

In [61]:
df = pd.read_csv('reviews/Newcardekho.csv', names=["reviews", "ratings", "attributes"])

In [66]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop = True, inplace=True)

In [63]:
df["reviews"] = df["reviews"].apply(lambda x: '' if x == "Read More" else x)
df["reviews"] = df["reviews"].apply(lambda x: np.nan if re.findall("selenium", x) else x)

,reviews,ratings,attributes
0,Nice car with good build quality and safety. P...,5.0,mileage
1,"Nice performance, full comfort in the car. Ver...",5.0,mileage
2,It is a very good car as compared to other EVs...,5.0,mileage
3,,4.0,mileage
4,Really happy to have Tata Nexon EV. It gives g...,5.0,mileage
...,...,...,...
143,,5.0,parts
144,,4.5,sensors
145,,2.0,showroom
146,,4.5,vent


['selenium', 'selenium']